<h1> ANN: Artificial Neural Networks </h1>
<br>
<h4> What are ANNs ? </h4>
<p>Artificial neural networks, usually simply called neural networks or neural nets, are computing systems inspired by the biological neural networks that constitute animal brains.</p>

<img align='left' width='500' src='https://www.tutorialspoint.com/artificial_intelligence/images/atypical_ann.jpg'>
<img align='right' width='500' src='https://miro.medium.com/max/1400/1*rptqFUQTMiruWZWKm-WMqQ.png'>


<p align ='right'> It contains input layer which is used to provide inputs to the neural networks.<br> This is then moved on to the hidden layer which computes various things.<br>And based on those computations we get the output layer<br>  Which neurons of the output layer is turned on determines where its classifed. In simple terms ANN is just a fancy classifier  </p>

<br>
<br>
<h4> Steps of ANN </h4>
<p> Here are simplified steps related to ANN which should be understood before coding </p>
<ol>
    <li> Initalize with random weights and bias </li>
    <li> Compute z for all input layers </li>
    <li> Feed z into activation function </li>
    <li> Procced to forward propagation on all layers </li>
    <li> Compute the cost function /loss function and Back propagate </li>
    <li> During Back Propagate adjusts weights and biases </li>
    <li> Repeat steps 2 to 6 untill n epochs </li>
</ol>
<h2> Project 1. Plant type predictor (Perceptron) </h2>

<h3> 0. Installing Dependencies </h3>

In [ ]:
!pip install pandas
!pip install numpy

<h3> 1. Importing Dependencies </h3>

In [ ]:
import pandas as pd
import random
import numpy as np 
import matplotlib.pyplot as plt

<h3> 2. Collecting Data set </h3>

In this data set their are lengths of leafs and widths of their leaves and their corresponding color patterns associated with it.


In [ ]:
df=pd.read_csv('Flower_dataset.csv')
df
features=np.array(df[['X1','X2']])
labels=np.array(df['Y']).reshape(-1,1)


<h3> 3. Perceptron Creation </h3>

<h4> 3.1 Plotting Graph to understand data </h4>

In [ ]:
plt.scatter(x=df['X1'],y=df['X2'])

<h4> 3.2 Coding Perceptron  </h4>

In [ ]:
def sigmoid(x):    
    return 1/(1+np.exp(-x))

def cost(P,A):
    return (P-A)**2

def d_cost(P,A):
    h=0.001
    return (cost(P+h,A)-cost(P,A))/h

def d_sigmoid(x):
    h=0.0001
    return (sigmoid(x+h)-sigmoid(x))/h




def train_model_perceptron(weights,bias,alpha,features,labels,epochs):
    for epoch in range(int(epochs)): 
        z=np.dot(features,weights)+bias
        P=[sigmoid(i) for i in z]
        for index in range(len(z)):
            weights=[weights[indx]-alpha*d_cost(P[index],labels[index][0])*d_sigmoid(z[index])*features[index][indx] for indx in range(len(weights))]    
            bias=bias-alpha*d_cost(P[index],labels[index][0])*d_sigmoid(z[index])
    return weights,bias

<h4> 3.3 Testing Perceptron </h4>

In [ ]:
weights=np.array([0.25,0.65])
bias=0.25
alpha=0.001

weights,bias=train_model_perceptron(weights,bias,alpha,features,labels,3500)
input_val=[float(i) for i in (input('Enter value of length and width seperate with comma: ')).split(',')]
print(input_val)
z=np.dot(input_val,weights)+bias
output=sigmoid(z)
if output>=0.50:
    output=1 
else: 
    output=0
print(output)

#See where our point lies at the end 
plt.scatter(x=df['X1'],y=df['X2'])
plt.scatter(x=input_val[0],y=input_val[1],color='red')
#Assignment plot the hypothetical line that is sperating 2 classes linear seperator.
plt.show()

<h2> Project 2. XOR Gate Predictor (Any dimension) </h2>
Inially we try to build the XOR Predictor using Preceptron then we move on to coding Neural Network with tensorflow and applying it on the same problem 
<br>

<h3> 1. Create Dataset for XOR Gate  </h3>
Let's create a dataset for any number of input XOR Gate 

In [ ]:
def generate_dataset(dims):

    def xor_output(x):
        return x%2


    df=pd.DataFrame()
    for i in range(0,dims):
        while True: 
            A=[random.randrange(0,2) for i in range(0,2**dims)]
            if A.count(1)==(2**dims)/2:
                break
        df[str(i)]=A

    op=0
    for i in df.columns:
        op+=df[i]

    df['Y']=op
    df['Y']=df['Y'].apply(xor_output)
    df
    return df 

dims=int(input('Enter number of dims: '))
data=generate_dataset(dims)
data

<h3> 2. Perceptron Approach </h3>
    <p>  we take input and then multiple with respective weights and biases </p>

In [ ]:
def generate_weights(data):
    print(data.columns)
    n=len(data.columns)-1
    return [random.uniform(0,1) for i in range(n)],random.uniform(-1,1)


def getting_proper_data(data,weights,bias):
    features=np.array(data.drop(columns=['Y']))
    labels=np.array(data['Y']).reshape(-1,1)
    weights=np.array(weights)
    bias=np.array(bias)
    return features,labels,weights,bias

alpha=10**-3
weights,bias=generate_weights(data)
features,labels,weights,bias=getting_proper_data(data,weights,bias)
weights,bias=train_model_perceptron(weights,bias,alpha,features,labels,3500)

In [ ]:
print(weights)
print(bias)

print('\nPredictor: ')
op=sigmoid(np.dot(weights,[1,1,1])+bias)
print(op)


<h4> Conclusion Perceptron </h4>
Here we can see that perceptron is not working well for XOR Gate mostly because it is not a linearly seperable problem we should hence not use perceptron in every possible cases and can use neural nets with hidden layers to handle the situation 

<h3>4. Tensorflow development </h3>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models

model=models.Sequential()
model.add(layers.Dense(64,activation='relu',input_dim=(data.drop(columns=['Y'])).shape[1]))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(128*2,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='sgd',metrics='accuracy')

X=np.array(data.drop(columns=['Y']))
Y=np.array(data['Y']).reshape(-1,1)
model.fit(X,Y,epochs=300)




<h4> Testing Tensorflow model </h4>

In [ ]:
print(model.predict([[1,0,0]]))

<h4> Conclusion Tensorflow NN model </h4>
As we can see that during this we were able to properly implement the Neural net capable of fitting and predicting in this case.